#LESSON 8: SECURE FEDERATED LEARNING FOR DEEPLEARNING PROJECT 


**Author** 
: Ateniola Oluwatobi Victor

**Objective** : My implementation of the Final project in the Secure Federated Learning
section of the Secure and Private AI Scholarship Challenge Nanodegree Program using the MNIST digit dataset

In [0]:
!pip install syft

     |████████████████████████████████| 256kB 2.8MB/s 
     |████████████████████████████████| 204kB 44.4MB/s 
     |████████████████████████████████| 389kB 42.7MB/s 
     |████████████████████████████████| 460kB 45.5MB/s 
     |████████████████████████████████| 256kB 42.5MB/s 
     |████████████████████████████████| 81kB 24.7MB/s 
     |████████████████████████████████| 1.4MB 45.7MB/s 
     |████████████████████████████████| 51kB 19.0MB/s 
     |████████████████████████████████| 266kB 52.3MB/s 
     |████████████████████████████████| 122kB 41.8MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067070 sha256=52b34b3210c9d4fb21ed25590756299eb33b39a71395651cedce944fd12088bd
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=2867023dda50fea2fcd0562b3cef8b295235920739a659cbf8eccade7e15ac7a
  Stored in di

In [0]:
import torch as th
import numpy as np
from torchvision import datasets, transforms
import torchvision.datasets as datasets
from torch.utils.data import Subset
from torch import nn
import torch.nn.functional as F
from torch import optim
import syft as sy
import helper

#Hooking syft to torch
hook = sy.TorchHook(th)

W0811 16:45:25.221754 139801331033984 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0811 16:45:25.240909 139801331033984 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
#Method to create 10 virtual workers and move to a list of workers
def create_workers():
  workers = []
  cartman = sy.VirtualWorker(hook, id = "cartman")
  workers.append(cartman)
  kyle = sy.VirtualWorker(hook, id = "kyle")
  workers.append(kyle)
  kenny = sy.VirtualWorker(hook, id = "kenny")
  workers.append(kenny)
  stan = sy.VirtualWorker(hook, id = "stan")
  workers.append(stan)
  butters = sy.VirtualWorker(hook, id = "butters")
  workers.append(butters)
  wendy = sy.VirtualWorker(hook, id = "wendy")
  workers.append(wendy)
  heidi = sy.VirtualWorker(hook, id = "heidi")
  workers.append(heidi)
  bebe = sy.VirtualWorker(hook, id = "bebe")
  workers.append(bebe)
  nichole = sy.VirtualWorker(hook, id = "nichole")
  workers.append(nichole)
  patty = sy.VirtualWorker(hook, id = "patty")
  workers.append(patty)
  return workers


In [0]:
#Method to clear out every tensor stored in the list of virtual workers
def clear_workers(workers):
  for worker in workers:
    worker.clear_objects()

In [0]:
#Method to split the mnist test dataset into the various workers and also to load the mnist test dataset into a test loader
def create_federated_and_test_loaders(workers, trainset, testset):
  federated_train_loader = sy.FederatedDataLoader(
      trainset.federate(workers), 
      batch_size=32, shuffle=True)

  test_loader = th.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
  return federated_train_loader, test_loader

In [0]:
#Method to bind remote optimizer functions to the Initial states of the remote models 
def create_models(workers, lr):
  remoteModels = list()
  remoteOptimizers = list()
  for worker in workers:
    model = classifier()
    model = model.send(worker)
    remoteOptimizers.append(optim.SGD(model.parameters(), lr))
    remoteModels.append(model)
  return remoteModels, remoteOptimizers
  

In [0]:
#Method to train models on the virtual workers without moving any gradients to the central model until the gradients have been collated.
def create_train_federated_models(workers, loader, lr = 0.12, epoch = 5):
  #sends model to first virtual worker
  virtual_models, virtual_optimizers = create_models(workers, lr)
#   virtual_model = classifier().send(workers[0])
#   optimizer = optim.SGD(virtual_model.parameters(), lr)
  criterion = nn.NLLLoss()
  for n in range(epoch):
    
    #Integer to keep up with first index.
    i = 0
    
    #Integer to keep up with current worker while training
    j = 0
    
    #Integer to count number of mini-batches per worker
    n_mbatch = 0
    
    #Variable to keep up with current worker while looping
    dbLoc = None
    
    #Variable to store cummulative loss.
    cum_loss = 0
    
    
    for batch_idx, (imgs, labels) in enumerate(loader):
      
      #condition to set dbLoc to the first worker
      if i == 0:
        i = 2
        dbLoc = imgs.location
        
      #condition to change dbLoc if img is stored on a different worker and also calculate loss
      if dbLoc is not imgs.location:
        print("The total loss for {0} for epoch {2} is {1}".format(workers[j].id, cum_loss / n_mbatch, n+1))
        dbLoc = imgs.location
        j += 1
        
        #Moving the model to a new worker
#         virtual_model.move(dbLoc)
        
        #Resetting the cummulative loss and batch count to zero for new worker
        cum_loss = 0
        n_mbatch = 0

      virtual_optimizers[j].zero_grad()
      output = virtual_models[j].forward(imgs)
      loss = criterion(output, labels)
      loss.backward()
      virtual_optimizers[j].step()
      cum_loss +=  loss.get().item()
      n_mbatch += 1
    print("The total loss for {0} is {1}".format(workers[j].id, cum_loss / n_mbatch))
  return virtual_models
  
    
  

In [0]:
#Method to return the model to the central machine
def create_central_model(model):
  return model.get()

In [0]:
#Method to convert the gradients of the remote models to their integar representation, encrypt and distribute among the provided workers
def share_gradients(models, workers):
  for model in models:
    model.fc1.weight.data = model.fc1.weight.data.fix_prec().share(*workers)
    model.fc1.bias.data = model.fc1.bias.data.fix_prec().share(*workers)
    model.fc2.weight.data = model.fc2.weight.data.fix_prec().share(*workers)
    model.fc2.bias.data = model.fc2.bias.data.fix_prec().share(*workers)
    model.fc3.weight.data = model.fc3.weight.data.fix_prec().share(*workers)
    model.fc3.bias.data = model.fc3.bias.data.fix_prec().share(*workers)
    model.fc4.weight.data = model.fc4.weight.data.fix_prec().share(*workers)
    model.fc4.bias.data = model.fc4.bias.data.fix_prec().share(*workers)
    model.fc5.weight.data = model.fc5.weight.data.fix_prec().share(*workers)
    model.fc5.bias.data = model.fc5.bias.data.fix_prec().share(*workers)   
  return models
  

In [0]:
#Method to aggregate the gradients accross the shared devices and return the parameters
def aggregate_grads(models):
  fc1_weight = list()
  fc1_bias = list()
  fc2_weight = list()
  fc2_bias = list()
  fc3_weight = list()
  fc3_bias = list()
  fc4_weight = list()
  fc4_bias = list()
  fc5_weight = list()
  fc5_bias = list()
  for model in models:
    fc1_weight.append(model.fc1.weight.data.clone().get())
    fc1_bias.append(model.fc1.bias.data.clone().get())
    fc2_weight.append(model.fc2.weight.data.clone().get())
    fc2_bias.append(model.fc2.bias.data.clone().get())
    fc3_weight.append(model.fc3.weight.data.clone().get())
    fc3_bias.append(model.fc3.bias.data.clone().get())
    fc4_weight.append(model.fc4.weight.data.clone().get())
    fc4_bias.append(model.fc4.bias.data.clone().get())
    fc5_weight.append(model.fc5.weight.data.clone().get())
    fc5_bias.append(model.fc5.bias.data.clone().get())
  params = {}
  params["fc1.weight"] = (sum(fc1_weight) / len(fc1_weight)).get().float_prec()
  params["fc1.bias"] = (sum(fc1_bias) / len(fc1_bias)).get().float_prec()
  params["fc2.weight"] = (sum(fc2_weight) / len(fc2_weight)).get().float_prec()
  params["fc2.bias"] = (sum(fc2_bias) / len(fc2_bias)).get().float_prec()
  params["fc3.weight"] = (sum(fc3_weight) / len(fc3_weight)).get().float_prec()
  params["fc3.bias"] = (sum(fc3_bias) / len(fc3_bias)).get().float_prec()
  params["fc4.weight"] = (sum(fc4_weight) / len(fc4_weight)).get().float_prec()
  params["fc4.bias"] = (sum(fc4_bias) / len(fc4_bias)).get().float_prec()
  params["fc5.weight"] = (sum(fc5_weight) / len(fc5_weight)).get().float_prec()
  params["fc5.bias"] = (sum(fc5_bias) / len(fc5_bias)).get().float_prec()
  return params


In [0]:
#Method to update the central model with the aggregated parameters from the remote models
def update_central_model(params):
  model = classifier()
  model.fc1.weight.data = params["fc1.weight"]
  model.fc1.bias.data = params["fc1.bias"]
  model.fc2.weight.data = params["fc2.weight"]
  model.fc2.bias.data = params["fc2.bias"]
  model.fc3.weight.data = params["fc3.weight"]
  model.fc3.bias.data = params["fc3.bias"]
  model.fc4.weight.data = params["fc4.weight"]
  model.fc4.bias.data = params["fc4.bias"]
  model.fc5.weight.data = params["fc5.weight"]
  model.fc5.bias.data = params["fc5.bias"]  
  return model

In [0]:
#Classifier for creating the models
class classifier(nn.Module):
  def __init__(self):
    super().__init__() 
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 64)
    self.fc4 = nn.Linear(64, 32)
    self.fc5 = nn.Linear(32, 10)
    
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = F.log_softmax(self.fc5(x), dim = 1)   
    return x
 

In [0]:
# Application of transforms to normalize the mnist data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [0]:
workers = create_workers()
clear_workers(workers)

In [0]:
federatedset, testset= create_federated_and_test_loaders(workers, mnist_trainset, mnist_testset)

In [0]:
remoteModels = create_train_federated_models(workers, federatedset, lr = 0.12, epoch = 6)

The total loss for cartman for epoch 1 is 1.888713796405082
The total loss for kyle for epoch 1 is 1.9061656721094822
The total loss for kenny for epoch 1 is 1.8456643531297117
The total loss for stan for epoch 1 is 1.9410718092893033
The total loss for butters for epoch 1 is 1.9659484123930018
The total loss for wendy for epoch 1 is 1.890808604499127
The total loss for heidi for epoch 1 is 1.769944117741382
The total loss for bebe for epoch 1 is 1.790674144283254
The total loss for nichole for epoch 1 is 1.8149789790523814
The total loss for patty is 1.871158241591555
The total loss for cartman for epoch 2 is 0.8731461511330402
The total loss for kyle for epoch 2 is 0.8791234556664812
The total loss for kenny for epoch 2 is 0.875950120785769
The total loss for stan for epoch 2 is 0.8649684112281242
The total loss for butters for epoch 2 is 0.9783720545312191
The total loss for wendy for epoch 2 is 0.9612294272222417
The total loss for heidi for epoch 2 is 0.9468026573353625
The total 

In [0]:
remoteModels = share_gradients(remoteModels, workers[5:8])


In [0]:
params  = aggregate_grads(remoteModels)

In [0]:
central_model = update_central_model(params)
